In [17]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import normalize
from sklearn import preprocessing
import geopandas as gpd
import folium
from folium import plugins
from datetime import timedelta, date
import shapely
from shapely.geometry import Polygon, Point
from matplotlib import dates as dt
from shapely.wkt import loads
import networkx as nx
from heapq import nsmallest
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
from bokeh.io import output_notebook, show
from ndlib.viz.bokeh.DiffusionTrend import DiffusionTrend

In [3]:
complete_info = pd.read_excel('data/network_edat_closest_n_v2.xlsx')
complete_info.head()

,Unnamed: 0,CODIABS,Edat,Educacio,Genere,Latitud,Longitud,Ocupacio,PNumContactes,color,id1,id2
0,0,383,2,1,0,41.364037,2.138546,3,2,red,399,791
1,0,383,2,1,0,41.364037,2.138546,3,2,red,399,232
2,1,383,3,2,0,41.367048,2.139810,6,2,red,1310,1198
3,1,383,3,2,0,41.367048,2.139810,6,2,red,1310,44
4,2,383,3,3,1,41.364385,2.139745,6,8,red,1530,811


In [4]:
barcelona_generated_dataframe = complete_info.copy()

In [5]:
def draw(viz):
    viz.plot()
    #fig, ax = plt.subplots(figsize=(9.5, 6.5))
    fig = plt.gcf()
    fig.canvas.toolbar_position = 'top'
    fig.set_size_inches(9, 4)
    fig.tight_layout()
    
    ax = plt.gca()
    ax.title.set_fontsize(12)
    ax.xaxis.label.set_fontsize(10)
    ax.yaxis.label.set_fontsize(10)
    ax.legend(fontsize=10);

In [6]:
def heat(ubi,information):#lat,lon,heat
    heat_ = []
    for iteration, row in information.iterrows():
        #### infected
        heat = []
        infected_nodes = row['Infected']
        try:
            for n in infected_nodes:
                lat = ubi.loc[n,'Latitud']
                long = ubi.loc[n,'Longitud']
                heat.append([lat,long,1])
            #### exposed
            exposed_nodes = row['Exposed']
            for n in exposed_nodes:
                lat = ubi.loc[n,'Latitud']
                long = ubi.loc[n,'Longitud']
                heat.append([lat,long,0.4])
            #### removed
            removed_nodes = row['Removed']
            for n in removed_nodes:
                lat = ubi.loc[n,'Latitud']
                long = ubi.loc[n,'Longitud']
                heat.append([lat,long,0.65])
        except:
            pass
        #### susceptible

        heat_.append(heat)
    return heat_

In [7]:
def plot(heat_list,location,zoom,rati,data):
    mapa = folium.Map(location = location,#[41.723864,1.901103],
                         zoom_start=zoom,#7.5,
                         ratio = rati,# 40,
                         tiles = "cartodbpositron")
    
    gradient = {.4: 'blue', .65: 'lime', 1: 'red'} #blau:exposats / verd: removed / vermell: infectats
    heatmap = plugins.HeatMapWithTime(
        heat_list,
        #index=list(data.iteration.unique()),
        name="Timelapse",
        auto_play=False,
        max_opacity=0.8, 
        gradient= gradient
    )
    heatmap.add_to(mapa)
    # Layercontrols lets you change visuals in the html page
    ctrl = folium.LayerControl()
    ctrl.add_to(mapa)
    mapa.save('files_bi/timelapse_seir.html')
    return mapa

In [8]:
def plotDot(row):
    dot = folium.CircleMarker(location=[row.Latitud, row.Longitud],
                        radius=2,
                        weight=3.5,color = row.color)
    dot.add_to(this_map)
    
def get_neigh_points(node):
    targets = network[network.id1 == node]['id2']
    neighs = []
    try:
        for t in targets:
            p = barcelona_generated_dataframe[barcelona_generated_dataframe.id1 == t].to_dict('records')[0]
            #get coords 
            coords = [p['Latitud'],p['Longitud']]
            neighs.append(coords)
    except:
        pass
    return neighs
    
def plotLine(row):
    #get source coords
    nodeSource = row.id1
    source_coords = [row.Latitud,row.Longitud]
    neighbors = get_neigh_points(nodeSource)
    #print(neighbors)
    for n in neighbors:
        t = [tuple(source_coords),tuple(n)]
        
        try:
            folium.PolyLine(t, color="lightgrey", weight=1, opacity=0.3).add_to(this_map)
        except:
            print(n)


In [9]:
network = pd.read_excel('network_closest_n.xlsx')
network.head()

,Unnamed: 0,CODIABS,Edat,Educacio,Genere,Ocupacio,PNumContactes,id1,id2
0,679,383,2,1,0,3,2,399,1198
1,679,383,2,1,0,3,2,399,791
2,1384,383,3,2,0,6,2,1310,325
3,1384,383,3,2,0,6,2,1310,1198
4,60,383,3,3,1,6,8,1530,409


In [10]:
network.drop(columns=['Unnamed: 0'],inplace=True)

## Llegim la informació espacial dels nodes:

In [11]:
ubi = complete_info[['id1','Latitud','Longitud']]
ubi = ubi.set_index(['id1'])
ubi = ubi.drop_duplicates()
ubi.head()

,Latitud,Longitud
id1,,
399,41.364037,2.138546
1310,41.367048,2.139810
1530,41.364385,2.139745
1183,41.365570,2.140459
121,41.361939,2.140670


## Creem la xarxa

In [12]:
cnetwork = nx.from_pandas_edgelist(network,'id1','id2', create_using=nx.MultiGraph)

for ind,data in network.iterrows():
        cnetwork.add_node(data.id1 ,edat = data.Edat, educacio = data.Educacio, ocupacio = data.Ocupacio)


#nx.draw_kamada_kawai(cnetwork, node_size=400, with_labels=True,edge_color="gray", font_color='yellow')

## Models epidemiològics

### SEIR:

- beta: prob infecció 
- gamma: probabilitat de removed 
- alpha: periode incubacio

R: INMUNE OR DECEASED --> VACUNACIÓ!!

SEIR assumes that if, during a generic iteration, a susceptible node comes into contact with an infected one, it becomes infected after an exposition period with probability beta, than it can switch to removed with probability gamma (the only transition allowed are S→E→I→R).

This implementation assumes discrete time dynamics for the E->I and I->R transitions.

In [62]:
network

,CODIABS,Edat,Educacio,Genere,Ocupacio,PNumContactes,id1,id2
0,383,2,1,0,3,2,399,1198
1,383,2,1,0,3,2,399,791
2,383,3,2,0,6,2,1310,325
3,383,3,2,0,6,2,1310,1198
4,383,3,3,1,6,8,1530,409
...,...,...,...,...,...,...,...,...
5112,396,2,3,1,6,3,625,640
5113,396,2,3,1,6,3,625,1397
5114,396,2,3,0,6,3,648,228
5115,396,2,3,0,6,3,648,1202


In [13]:
# Model Selection
model = ep.SEIRModel(cnetwork)

# Model Configuration __> A social network model of COVID-19
config = mc.Configuration()
config.add_model_parameter('beta', 0.5)    # infection rate
config.add_model_parameter('gamma', 0.2)    # recovery rate
config.add_model_parameter("fraction_infected", 0.001)
config.add_model_parameter('alpha', 1/5.2) ## removal prob

config.add_model_parameter('alpha', 1/5.2)
model.set_initial_status(config)


In [52]:
model.available_statuses

{'Susceptible': 0, 'Exposed': 2, 'Infected': 1, 'Removed': 3}

In [53]:
N = 100
information = pd.DataFrame(columns=['iteration','Susceptible','Infected','#Infected','Exposed','#Exposed','Removed','#Removed'])


for i in range(1,N+1):
    info_model = model.iteration(i)
    row = pd.DataFrame(columns=['iteration','Susceptible','Infected','Exposed','Removed'])
    #print('############# Iteració nº '+ str(i))
    
    information.at[i,'iteration'] = info_model['iteration']
    
    status = info_model['status']
    information.at[i,'Susceptible'] = [node for node, status in status.items() if status == 0]
    information.at[i,'##Susceptible'] = len([node for node, status in status.items() if status == 0])
    
    information.at[i,'Infected'] = [node for node, status in status.items() if status == 1]
    information.at[i,'#Infected'] = len([node for node, status in status.items() if status == 1])
    
    
    information.at[i,'Exposed'] = [node for node, status in status.items() if status == 2]
    information.at[i,'#Exposed'] = len([node for node, status in status.items() if status == 2])
    information.at[i,'Removed'] = [node for node, status in status.items() if status == 3]
    information.at[i,'#Removed'] = len([node for node, status in status.items() if status == 3])

    information.append(row)

In [60]:
heat_list =  heat(ubi,information)

In [61]:

plot(heat_list,location = [41.4031763,2.1317536],#[41.723864,1.901103],
                     zoom = 12,
                      rati = 20,
                      data = information
                     )

In [95]:
model.get_info()

{'beta': 0.5,
 'gamma': 0.2,
 'fraction_infected': 0.05,
 'alpha': 0.1923076923076923,
 'tp_rate': 1}

In [14]:
iterations = model.iteration_bunch(200)
trends = model.build_trends(iterations)

100%|██████████| 200/200 [00:05<00:00, 34.07it/s]


In [15]:
output_notebook()
viz = DiffusionTrend(model, trends)
p = viz.plot(width=800, height=600)
show(p)

Loading BokehJS ...

In [70]:
N = 10
information = pd.DataFrame(columns=['iteration','status','node_count','status_delta'])
for i in range(N):
    iteration = model.iteration(i)
    print('############# Iteració nº '+ str(i))
    print(model.get_status_map())
    print(iteration)

############# Iteració nº 0
{'Susceptible': 0, 'Exposed': 2, 'Infected': 1, 'Removed': 3}
{'iteration': 231, 'status': {}, 'node_count': {0: 5, 2: 0, 1: 0, 3: 1605}, 'status_delta': {0: 0, 2: 0, 1: 0, 3: 0}}
############# Iteració nº 1
{'Susceptible': 0, 'Exposed': 2, 'Infected': 1, 'Removed': 3}
{'iteration': 232, 'status': {}, 'node_count': {0: 5, 2: 0, 1: 0, 3: 1605}, 'status_delta': {0: 0, 2: 0, 1: 0, 3: 0}}
############# Iteració nº 2
{'Susceptible': 0, 'Exposed': 2, 'Infected': 1, 'Removed': 3}
{'iteration': 233, 'status': {}, 'node_count': {0: 5, 2: 0, 1: 0, 3: 1605}, 'status_delta': {0: 0, 2: 0, 1: 0, 3: 0}}
############# Iteració nº 3
{'Susceptible': 0, 'Exposed': 2, 'Infected': 1, 'Removed': 3}
{'iteration': 234, 'status': {}, 'node_count': {0: 5, 2: 0, 1: 0, 3: 1605}, 'status_delta': {0: 0, 2: 0, 1: 0, 3: 0}}
############# Iteració nº 4
{'Susceptible': 0, 'Exposed': 2, 'Infected': 1, 'Removed': 3}
{'iteration': 235, 'status': {}, 'node_count': {0: 5, 2: 0, 1: 0, 3: 1605}, 's